# Start 

Below is an example of what a user would have to create in order to get the LShaped.jl package to run their own custom scenario-based model. I use, for simplicity, an example out of Introduction to Stochastic Programming (2nd). This comes from Ch. 5, where the L-Shaped Method is introduced.

In [ ]:
using Pkg
Pkg.activate("..")

In [ ]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [ ]:
using LShaped

In [ ]:
#=names = ["n1","n2","n3"]

df = DataFrame()

nvars = 3

for i = 1:nvars
    insertcols!(df, i, Symbol(names[i])=>Float64[])
end

vals = [1,2,3]

# this is dangerous and requires a specific ordering of the values. very dangerous :O
push!(df, vals)=#

In [ ]:
#CSV.write("./test.csv", df)

In [ ]:
#=a = [4,5,6];

sa = string(a)
n = length(string(a))

sa = sa[2:n-1]
open("./test.csv", "a") do io
        write(io, "$(sa) \n")
end
=#

In [ ]:
#dfload = DataFrame(CSV.File("./test.csv"))

In [ ]:
#a = [4,5,6];

#CSV.write("./test.csv", a; append=true)

### Create subproblem generating function (dependent on scenario id)

In [ ]:
# implementation of Birge, Louveax Ch. 5 Ex 1 Second Stage

function subproblem_constructor(sid)
    
    q1 = [-24, -28]
    q2 = [-28, -32]
    d1 = [500, 300]
    d2 = [100, 300]
    
    model = Model(with_optimizer(Gurobi.Optimizer; OutputFlag=0))
    
    @variable(model, x1 )# >= 40.0)
    @variable(model, x2 )# >= 20.0)
    @variable(model, y1[sid] >= 0)
    @variable(model, y2[sid] >= 0)
    
    @objective(model, Min, 100*x1 + 150*x2 + q1[sid]*y1[sid] + q2[sid]*y2[sid])
    
   # @constraint(model, x1 + x2 <= 120)
    
    @constraint(model, 6*y1[sid] + 10*y2[sid] <= 60*x1)
    @constraint(model, 8*y1[sid] + 5*y2[sid] <= 80*x2)
    
    @constraint(model, y1[sid] <= d1[sid])
    @constraint(model, y2[sid] <= d2[sid])
    
    return model
    
end


### Create variable dictionary. Key 1 maps to stage 1 variables. Key 2 maps to stage 2 variables.

TODO: have it construct this inside of the package (e.g. only give instructions to do it)

In [ ]:
v_dict = Dict()

v_dict[1] = [("x1", 40, Inf, 40), ("x2", 20, Inf, 20)]
v_dict[2] = ["y1", "y2"]

## create first stage problem

In [ ]:
function create_first_stage()
  
    fs = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0));
    
    @variable(fs, x1 >= 40)
    @variable(fs, x2 >= 20)
    
    @objective(fs, Min, 100*x1 + 150*x2)
    
    @constraint(fs, x1 + x2 <= 120)
    
    return fs
    
end


### Run algorithm

In [ ]:
xn, firststage, fs = LShaped.L_Shaped_Algorithm(subproblem_constructor, 
                                        v_dict, 2, create_first_stage, 1e-6, 10, [0.4, 0.6]; store="./bl_data/");

In [ ]:
rm("bl_data",recursive=true)